In [ ]:
#处理时间较久，非一次完成，故会存储几个中间处理文件以防处理结果丢失

In [1]:

#提取出有id的数据，即在aminer_name_cluster_edit的id在dpv12的ven_id中的
import json
from tqdm import tqdm 

raw_id=[]
ven_id=[]
labels = ["0", "3", "4", "5", "6", "7", "8", "10", "11"]
label_to_class = {label: i for i, label in enumerate(labels)}
with open("data/aminer_name_cluster_edit.txt",'r')as fin1:   
    for items in tqdm(fin1):
        item=items.strip().split('\t')
        if item[-1] in label_to_class:
            ven_id.append(item[0])
print("number of venue ids", len(ven_id))

with open ("data/dblp.v12.json",'rb') as  fin:
    print("load dblp.v12.json...")
    raw=json.load(fin) 
    print("load done")
    i=0
    for index, d in tqdm(enumerate(raw)):
        try :
            if str(d.get('venue').get('id')) in ven_id:
                raw_id.append(d)
        except:
            i+=1
            pass

print("总数据数",index,"没有venue_id",i)
print("拥有venue_id的数据",len(raw_id))



440it [00:00, 941098.30it/s]


number of venue ids 396
load dblp.v12.json...
load done


4894081it [00:34, 143733.33it/s]

总数据数 4894080 没有venue_id 47108
拥有venue_id的数据 1316213


In [2]:
#提取出需要的信息，并过滤没有摘要的数据
new_data=[]
no_abs=0
for index,data in tqdm(enumerate(raw_id)):
    # if index==10000:
    #     break
    new_dic={}
    new_dic['id']=data.get('id')
    new_dic['title']=data.get('title')
    #获取摘要,摘要被压缩了，还原成正常摘要
    try:
        abs_length=data.get('indexed_abstract').get('IndexLength')
        abs_dic=data.get('indexed_abstract').get('InvertedIndex')
        abstract=[None for _ in range(abs_length)]
        for key,values in abs_dic.items():
            for i in values:
                abstract[i]=key
        str_abs=''
        for i in abstract:
            str_abs+=i+' '
        str_abs=str_abs.strip(' ')
        new_dic['abstract']=str_abs
    except:
        no_abs+=1
        
        continue

    #获取作者信息
    new_dic['authors']=data.get('authors')
    new_dic['references']=data.get('references')
    new_dic['fos']=data.get('fos')
    new_dic['year']=data.get('year')
    new_dic['venue']=data.get('venue')
    new_data.append(new_dic)
print("总的dpv12_venue_id.json带标签数据",len(raw_id),"处理完新的数据：",len(new_data),"没有摘要的文章",no_abs)
with open ("data/dpv12_venue_id_filter.json",'w')as fout3:
    json.dump(new_data,fout3,ensure_ascii=False)

1316213it [01:28, 14881.33it/s]


总的dpv12_venue_id.json带标签数据 1316213 处理完新的数据： 1098513 没有摘要的文章 217700


In [9]:
# 利用del_noref.py文件进行删除不存在的引用的数据，得到dpv12_venue_id_filter_delrefid.json
# Run del_noref.py to delete papers with non-existent references, and get dpv12_venue_id_filter_delrefid.json

1402160


In [3]:
from tqdm import tqdm 
import json
#下面利用venue_id给数据打标签为0-8
id_lab={}
with open('data/aminer_name_cluster_edit.txt','r')as fi1:
    for line in fi1:
        items=line.strip().split('\t')
        id=int(items[0])
        name=items[1]
        label=items[2]
        if label in label_to_class:
            id_lab[id]=label_to_class[label]

#之前数据中对应的标签有-1，cogdl处理得到的num_classe出错，在aminer_name_cluster_edit.txt直接替换改为18

with open("data/dpv12_venue_id_filter_delrefid.json",'r')as fp:
    data=json.load(fp)
    for index,da in enumerate(tqdm(data)):
        lid=da['venue']['id']
        da['label']=id_lab[lid]

#使用顺序索引来作为edge_index,添加index,references_index
id_index={}

for index,da in enumerate(tqdm(data)):
    id_index[da['id']]=index
    da['index']=index
    
for index,da in enumerate(tqdm(data)):
    ref=da['references']
    ref_list=[]
    if ref!=None:
        for item in ref:
            ref_index=id_index.get(item)
            ref_list.append(ref_index)
    da['references_index']=ref_list


with open('data/dpv12_last.json','w')as fout_1:
    data=json.dump(data,fout_1,ensure_ascii=False)

100%|██████████| 992606/992606 [00:07<00:00, 130557.21it/s]


In [4]:
import json
from tqdm import tqdm
with open('data/dpv12_last.json','r') as fi2:
    data=json.load(fi2)

In [5]:
#下面来抽取出graph，即图的引用关系

ref_list=[]

for index,da in enumerate(tqdm(data)):
    
    res=da['references_index']
    ref_dic={}
    if len(res)!=0:
        ref_dic['index']=da['index']
        ref_dic['references_index']=res
        ref_list.append(ref_dic)
cita=[]
with open('data/graph.txt','w')as fo2:
 
    for i in ref_list:
        id=str(i['index'])
        for j in i['references_index']:
            cita.append(str(j)+' '+id+'\n')
    fo2.writelines(cita)

100%|██████████| 992606/992606 [00:01<00:00, 795944.07it/s]


In [6]:
#根据年份划分训练集，测试集

train=[]
valid=[]
test=[]
for index,da in enumerate(tqdm(data)):
    year=da['year']
    if year<= 2012:
        train.append(da)
    elif year<=2014:
        valid.append(da)
    else:
        test.append(da)
print("train长度",len(train),"test",len(test),"valid",len(valid))
with open('data/dpv12_train.json','w')as fs2:
    json.dump(train,fs2,ensure_ascii=False)
with open('data/dpv12_valid.json','w')as fs2:
    json.dump(valid,fs2,ensure_ascii=False)
with open('data/dpv12_test.json','w')as fs2:
    json.dump(test,fs2,ensure_ascii=False)


#提取出y，ty,vy
y_train=[]
y_test=[]
y_valid=[]

for i in train:
    y_train.append(str(i['label'])+'\n')

for i in valid:
    y_valid.append(str(i['label'])+'\n')

for i in test:
    y_test.append(str(i['label'])+'\n')
with open('data/y.txt','w') as fo1:
    fo1.writelines(y_train)
with open('data/ty.txt','w') as fo1:
    fo1.writelines(y_test)
with open('data/vy.txt','w') as fo1:
    fo1.writelines(y_valid)

100%|██████████| 992606/992606 [00:00<00:00, 1112157.74it/s]


train长度 634273 test 250555 valid 107778
